In [4]:
%load_ext autoreload
%autoreload 2

from aiida import orm, engine, load_profile
from aiida.common import NotExistent

from aiida_dislocation.workflows.usf import USFWorkChain

from ase.io import read
from ase.build import bulk
from aiida.engine import run

import logging


load_profile()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Profile<uuid='155a05c5c8f7420aa49bbf9207908698' name='supercon'>

In [5]:
prepend_text = """
module load intel
module load mkl
module load impi
module load fftw/3.3.10
"""

group_label = 'test/workchains/USF/'
test_USF_group, created = orm.Group.collection.get_or_create(label=group_label)
if created:
    print(f"Group '{group_label}' created.")
else:
    print(f"Group '{group_label}' already exists.")


computer = 'mn5'
mn5_computer = orm.load_computer(computer)
mn5_computer.set_workdir('/gpfs/scratch/ehpc14/uclo653525/aiida_workdir/test_aiida_dislocation')

version = '730'
pw_label = f'qe-{version}-pw'
pw_code = orm.load_code(f'{pw_label}@{computer}')


Group 'test/workchains/USF/' already exists.


# TEST A1 -- Aluminium

In [6]:
import yaml

ase_atom = read('./structures/cif/Al-225-A1.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([12, 12, 12])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'nowf'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.02
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(9)
inputs['slipping_system'] = orm.List(['A1', '111', None])


In [7]:
logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)
test_USF_group.add_nodes(test_node.node)

04/28/2025 09:18:10 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|validate_slipping_system]: We are doing FCC element crystal <space group #225, prototype Cu>. Usually, the gliding plane is 111.
04/28/2025 09:18:10 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|validate_slipping_system]: ABC stacking for {111} gliding plane. Faulting possible. slipping direction is along 1/2[110].
04/28/2025 09:18:10 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|run_scf]: launching PwCalculation<71585> for Al unit cell geometry.
04/28/2025 09:21:02 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|inspect_scf]: SCF calculation<71585> finished
04/28/2025 09:21:02 AM <85805> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [71584|USFWorkChain|run_USF]: Multiplicity of the faulted geometr

# TEST A2 -- Vanadium

In [9]:
import yaml

ase_atom = read('./structures/cif/V-227-A2.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([12, 12, 12])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'nowf'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.02
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5
inputs['skip_uc'] = orm.Bool(False)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['A2', '100', None])


In [10]:
test_node = run.get_node(USFWorkChain, **inputs)

04/26/2025 04:08:12 PM <1288690> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137844|USFWorkChain|validate_slipping_system]: We are doing A2 structure. <space group 227, prototype V>. I don't know the usual gliding plane.
04/26/2025 04:08:12 PM <1288690> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137844|USFWorkChain|validate_slipping_system]: AB stacking for {100} gliding plane. Faulting possible. slipping direction is along 1/2[110].
04/26/2025 04:08:13 PM <1288690> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137844|USFWorkChain|run_scf]: launching PwCalculation<1137845> for V unit cell geometry.
04/26/2025 04:08:25 PM <1288690> aiida.transport.SshTransport: [WARNING] There was nonempty stderr in the whoami command: Set INTEL compilers as MPI wrappers backend
load impi/2021.10.0 (PATH, MANPATH, LD_LIBRARY_PATH)
load mkl/2023.2.0 (LD_LIBRARY_PATH)
load UCX/1.15.0 (PATH, LD_LIBRARY_PATH, LIBRARY_PATH, C_INCLUDE_P

IndexError: tuple index out of range

# TEST HALF-HEUSLER

In [11]:

# structure_grp = 'paper/structures/candidate'
# query = orm.QueryBuilder()
# query.append(
#     orm.Group,
#     filters={"label": structure_grp},
#     tag="group"
#     ).append(
#     orm.StructureData,
#     with_group='group',
#     project=["*", "extras.source_db", "extras.source_id"],
#     )

# structures = {
#     f'{source_db}-{source_id}': structure for structure, source_db, source_id in query.iterall()
# }

# structures['mpds-S1605628'].export('/home/yzhang/project/lucia/git/aiida-dislocation/src/aiida_dislocation/examples/structures/mpds-S1605628-AsTe-225-B1.xyz')
# structures['mpds-S1612207'].export('/home/yzhang/project/lucia/git/aiida-dislocation/src/aiida_dislocation/examples/structures/mpds-S1612207-NbCoSb-216-C1b.xyz')
# structures['icsd-77392'].export('/home/yzhang/project/lucia/git/aiida-dislocation/src/aiida_dislocation/examples/structures/icsd-77392-TaRu3C-221-E21.xyz')
# structures['mpds-S378780'].export('/home/yzhang/project/lucia/git/aiida-dislocation/src/aiida_dislocation/examples/structures/mpds-S378780-Nb3Sn-223-A15.xyz')
# structures['icsd-659260'].export('/home/yzhang/project/lucia/git/aiida-dislocation/src/aiida_dislocation/examples/structures/icsd-659260-IrS2-205-C2.xyz')

import yaml

ase_atom = read('./structures/cif/mpds-S1612207-NbCoSb-216-C1b.cif', format='cif')
structure = orm.StructureData(ase=ase_atom)

pseudo_family = orm.load_group('PseudoDojo/0.5/PBE/SR/standard/upf')

pseudos = pseudo_family.get_pseudos(structure=structure)

cutoff_wfc, cutoff_rho = pseudo_family.get_recommended_cutoffs(
    structure=structure,
    unit='Ry'
)

kpoints = orm.KpointsData()
kpoints.set_kpoints_mesh([12, 12, 12])

settings = orm.Dict({
    # 'cmdline': ['-npool', '2']
})

usf_overrides_yaml = open('./usf.yaml')
usf_overrides = yaml.safe_load(usf_overrides_yaml)

inputs = {}

inputs['metadata'] = usf_overrides.get('metadata', {})
inputs['structure'] = structure
inputs['kpoints'] = kpoints
inputs['pw'] = usf_overrides.get('pw', {})
inputs['pw']['pseudos'] = pseudos
inputs['pw']['code'] = pw_code
inputs['pw']['parameters']['CONTROL']['disk_io'] = 'minimal'
inputs['pw']['parameters']['SYSTEM']['degauss'] = 0.01
inputs['pw']['parameters']['ELECTRONS']['conv_thr'] = 1e-5
inputs['skip_uc'] = orm.Bool(True)
inputs['n_layers'] =orm.Int(8)
inputs['slipping_system'] = orm.List(['C1b', '110', None])



In [12]:
import logging

logging.getLogger('aiida.transport.SshTransport').setLevel(logging.ERROR)
logging.getLogger('aiida.scheduler.slurm').setLevel(logging.ERROR)

test_node = run.get_node(USFWorkChain, **inputs)

04/26/2025 12:11:00 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137727|USFWorkChain|validate_slipping_system]: We are doing half-heusler-type structure, space group (216).
04/26/2025 12:11:00 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137727|USFWorkChain|validate_slipping_system]: AB stacking for {110} gliding plane. Faulting possible.slipping direction is along 1/2[-111].
04/26/2025 12:11:00 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [WARNING] You are skipping the unitcell calculation.Hopefully you know what you are doing!
04/26/2025 12:11:00 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137727|USFWorkChain|run_USF]: Multiplicity of the faulted geometry: uuid: 5bc1e3a8-792c-4a50-94e7-53ed01b2253d (unstored) value: 8.0
04/26/2025 12:11:01 PM <1225033> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [1137727|USFWorkChain|run_USF]